# Tâche #3 : Classification d'incidents avec des modèles *Transformers*

Toujours avec la même tâche et les mêmes fichiers de textes, utiliser la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle bert-base-uncased et un autre modèle de votre choix. 
Les consignes associées à cette tâche sont: 
-	Nom du notebook : transformer.ipynb
-	Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers. 
-	Plongements de mots : Ceux du modèle transformer. 
-	Normalisation : Lettre en minuscule pour Bert. Aucune contrainte pour le 2e modèle. 
-	Choix du 2e transformer: Un modèle encodeur préentraîné pour l’anglais. Le modèle ne doit pas être une autre version de Bert et doit être significativement différent. Utilisez un 2 fichier pour ce modèle si nécessaire (une copie de celui-ci). 
-	Analyse : Comparer les résultats obtenus avec les 2 modèles transformers. Présentez également une comparaison globale des résultats obtenus avec tous les modèles utilisés dans ce travail et ceux du travail précédent (TP #1). 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexe. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Création du jeu de données (*dataset*)

In [1]:
import spacy

spacy_model = spacy.load("en_core_web_md")
embedding_size = spacy_model.meta['vectors']['width']


import pandas as pd
import json
import numpy as np
# Assurez-vous que le modèle de langue de spacy est téléchargé
# python -m spacy download fr_core_news_md (par exemple pour le français)

# Charger le modèle de langue de spacy

# Définition des chemins vers les fichiers de données
train_data_path = './data/incidents_train.json'
dev_data_path = './data/incidents_dev.json'
test_data_path = './data/incidents_test.json'

def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
        
        text = [item["text"] for item in incident_list]
        target = np.array([int(item["label"]) for item in incident_list])
         
    return text, target


# Créer les DataFrames pour chaque partition de données
train_list, train_target = load_incident_dataset(train_data_path)
dev_list, dev_target = load_incident_dataset(dev_data_path)
test_list, test_target = load_incident_dataset(test_data_path)

# Affichage de l'information de base sur les DataFrames
display(f"Train data: text_size {len(train_list)}, target_size {len(train_target)}")
display(f"Dev data: text_size {len(dev_list)}, target_size {len(dev_target)}")
display(f"Test data: text_size {len(test_list)}, target_size {len(test_target)}")



# Vérification des premiers enregistrements dans l'ensemble d'entraînement
train_list[0]


'Train data: text_size 2475, target_size 2475'

'Dev data: text_size 531, target_size 531'

'Test data: text_size 531, target_size 531'

' At approximately 8:50 a.m. on October 29  1997  Employee #1 was painting a  single story house at 2657 7th Ave  Sacramento  CA. He was caulking around the  peak of the roof line on the west side of the house  20 ft above the ground.  He was working off of a 24 ft aluminum extension ladder so that his feet were  approximately 12 to 13 feet above the ground. Employee #1 fell and suffered a  concussion and two dislocated discs in his lower back and was hospitalized.  The ladder was not secured to prevent movement.                                 '

In [2]:
# import numpy as np

# padding_token = "<PAD>"   # mot 0
# unk_token = "<UNK>"    # mot 1
# zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

# id2word = {}
# id2word[0] = padding_token 
# id2word[1] = unk_token 

# word2id = {}
# word2id[padding_token] = 0
# word2id[unk_token] = 1

# id2embedding = {}
# id2embedding[0] = zero_vec_embedding
# id2embedding[1] = zero_vec_embedding

## 2. Création de modèle(s)

/Users/mac/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bert

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizer, BertModel, AutoTokenizer, AutoModel, DataCollatorWithPadding, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(text_list, labels):
    return tokenizer(text_list, truncation=True, padding=True, return_tensors="pt"), labels

tokenized_train_list = [preprocess_function(text, labels) for text, labels in zip(train_list, train_target)]
tokenized_validate_list = [preprocess_function(text, labels) for text, labels in zip(dev_list, dev_target)]
tokenized_test_list = [preprocess_function(text, labels) for text, labels in zip(test_list, test_target)]


print(tokenized_train_list[0])


bert_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=9)

({'input_ids': tensor([[  101,  2012,  3155,  1022,  1024,  2753,  1037,  1012,  1049,  1012,
          2006,  2255,  2756,  2722,  7904,  1001,  1015,  2001,  4169,  1037,
          2309,  2466,  2160,  2012, 20549,  2581,  5504, 13642, 11932,  6187,
          1012,  2002,  2001,  6187,  5313,  6834,  2105,  1996,  4672,  1997,
          1996,  4412,  2240,  2006,  1996,  2225,  2217,  1997,  1996,  2160,
          2322,  3027,  2682,  1996,  2598,  1012,  2002,  2001,  2551,  2125,
          1997,  1037,  2484,  3027, 13061,  5331, 10535,  2061,  2008,  2010,
          2519,  2020,  3155,  2260,  2000,  2410,  2519,  2682,  1996,  2598,
          1012,  7904,  1001,  1015,  3062,  1998,  4265,  1037, 23159,  1998,
          2048,  4487, 14540, 24755,  3064, 15303,  1999,  2010,  2896,  2067,
          1998,  2001, 24735,  1012,  1996, 10535,  2001,  2025,  7119,  2000,
          4652,  2929,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:


# custom_model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=9
# )

## 3. Entraînement de modèle(s)

Bert Training

In [10]:
import evaluate

accuracy = evaluate.load("glue", "mrpc")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
import torch.nn as nn
import torch

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Get the model's classification layer weights
        classification_layer_weights = model.classifier.weight
        
        # Assuming you want to use the weights directly, you can pass them to the loss function
        loss_fct = nn.CrossEntropyLoss(weight=classification_layer_weights)
        
        # Compute the loss
        loss = loss_fct(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

In [25]:
training_args = TrainingArguments(
    output_dir="model3/bert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
    
)

trainer = CustomTrainer(
    model=bert_model,
    args=training_args,
    train_dataset=list(zip(tokenized_train_list, train_target)),
    eval_dataset=list(zip(tokenized_validate_list, dev_target)),
    tokenizer=tokenizer,
    data_collator=bert_data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

KeyboardInterrupt: 

## 4. Évaluation et analyse de résultats